<a href="https://colab.research.google.com/github/dinagalevska/DrugCombLinkPrediction/blob/master/IIS_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import ast
from transformers import BertTokenizer, BertModel
import torch

In [ ]:
file_path = '/content/drive/MyDrive/INTELIGENTNI_INFORMACISKI_SISTEMI/raw_data.csv'

data = pd.read_csv(file_path)

In [ ]:
data.head()

,nctid,status,why_stop,label,phase,diseases,icdcodes,drugs,smiless,criteria
0,NCT00000172,completed,NaN,1,phase 3,['alzheimer disease'],"[""['G30.8', 'G30.9', 'G30.0', 'G30.1']""]",['galantamine'],['[H][C@]12C[C@@H](O)C=C[C@]11CCN(C)CC3=C1C(O2...,\n Inclusion Criteria:\r\n\r\n ...
1,NCT00000173,completed,NaN,1,phase 3,['alzheimer disease'],"[""['G30.8', 'G30.9', 'G30.0', 'G30.1']""]","['donepezil', 'vitamin e']",['O=S(=O)(C1=CC=CC=C1)C1=CN=C2C(C=CC=C2N2CCNCC...,\n Inclusion Criteria:\r\n\r\n ...
2,NCT00000174,completed,NaN,0,phase 3,"['alzheimer disease', 'cognition disorders']","[""['G30.8', 'G30.9', 'G30.0', 'G30.1']"", ""['F2...",['rivastigmine'],['CCN(C)C(=O)OC1=CC=CC(=C1)[C@H](C)N(C)C'],\n Inclusion Criteria:\r\n\r\n ...
3,NCT00000390,completed,NaN,1,phase 2,['depression'],"[""['F32.A', 'F53.0', 'P91.4', 'Z13.31', 'Z13.3...",['imipramine hydrochloride'],['CN(C)CCCN1C2=CC=CC=C2CCC2=CC=CC=C12'],\n Inclusion Criteria:\r\n\r\n ...
4,NCT00000419,terminated,NaN,0,phase 3,['systemic lupus erythematosus'],"[""['M32.9', 'M32.0', 'M32.11', 'M32.12', 'M32....",['premarin and provera'],['[H][C@@]12CC[C@](OC(C)=O)(C(C)=O)[C@@]1(C)CC...,\n Inclusion Criteria:\r\n\r\n ...


In [ ]:
data.drop(columns=['nctid', 'phase', 'icdcodes'], axis=1, inplace=True)

In [ ]:
data.head()

,status,why_stop,label,diseases,drugs,smiless,criteria
0,completed,NaN,1,['alzheimer disease'],['galantamine'],['[H][C@]12C[C@@H](O)C=C[C@]11CCN(C)CC3=C1C(O2...,\n Inclusion Criteria:\r\n\r\n ...
1,completed,NaN,1,['alzheimer disease'],"['donepezil', 'vitamin e']",['O=S(=O)(C1=CC=CC=C1)C1=CN=C2C(C=CC=C2N2CCNCC...,\n Inclusion Criteria:\r\n\r\n ...
2,completed,NaN,0,"['alzheimer disease', 'cognition disorders']",['rivastigmine'],['CCN(C)C(=O)OC1=CC=CC(=C1)[C@H](C)N(C)C'],\n Inclusion Criteria:\r\n\r\n ...
3,completed,NaN,1,['depression'],['imipramine hydrochloride'],['CN(C)CCCN1C2=CC=CC=C2CCC2=CC=CC=C12'],\n Inclusion Criteria:\r\n\r\n ...
4,terminated,NaN,0,['systemic lupus erythematosus'],['premarin and provera'],['[H][C@@]12CC[C@](OC(C)=O)(C(C)=O)[C@@]1(C)CC...,\n Inclusion Criteria:\r\n\r\n ...


In [ ]:
data['diseases'] = data['diseases'].apply(lambda x: ', '.join(eval(x)))
data['drugs'] = data['drugs'].apply(lambda x: ', '.join(eval(x)))

In [ ]:
data.head()

,status,why_stop,label,diseases,drugs,smiless,criteria
0,completed,NaN,1,alzheimer disease,galantamine,['[H][C@]12C[C@@H](O)C=C[C@]11CCN(C)CC3=C1C(O2...,\n Inclusion Criteria:\r\n\r\n ...
1,completed,NaN,1,alzheimer disease,"donepezil, vitamin e",['O=S(=O)(C1=CC=CC=C1)C1=CN=C2C(C=CC=C2N2CCNCC...,\n Inclusion Criteria:\r\n\r\n ...
2,completed,NaN,0,"alzheimer disease, cognition disorders",rivastigmine,['CCN(C)C(=O)OC1=CC=CC(=C1)[C@H](C)N(C)C'],\n Inclusion Criteria:\r\n\r\n ...
3,completed,NaN,1,depression,imipramine hydrochloride,['CN(C)CCCN1C2=CC=CC=C2CCC2=CC=CC=C12'],\n Inclusion Criteria:\r\n\r\n ...
4,terminated,NaN,0,systemic lupus erythematosus,premarin and provera,['[H][C@@]12CC[C@](OC(C)=O)(C(C)=O)[C@@]1(C)CC...,\n Inclusion Criteria:\r\n\r\n ...


In [ ]:
data['smiless'] = data['smiless'].apply(lambda x: ', '.join(eval(x)))

In [ ]:
data.head()

,status,why_stop,label,diseases,drugs,smiless,criteria
0,completed,NaN,1,alzheimer disease,galantamine,[H][C@]12C[C@@H](O)C=C[C@]11CCN(C)CC3=C1C(O2)=...,\n Inclusion Criteria:\r\n\r\n ...
1,completed,NaN,1,alzheimer disease,"donepezil, vitamin e",O=S(=O)(C1=CC=CC=C1)C1=CN=C2C(C=CC=C2N2CCNCC2)...,\n Inclusion Criteria:\r\n\r\n ...
2,completed,NaN,0,"alzheimer disease, cognition disorders",rivastigmine,CCN(C)C(=O)OC1=CC=CC(=C1)[C@H](C)N(C)C,\n Inclusion Criteria:\r\n\r\n ...
3,completed,NaN,1,depression,imipramine hydrochloride,CN(C)CCCN1C2=CC=CC=C2CCC2=CC=CC=C12,\n Inclusion Criteria:\r\n\r\n ...
4,terminated,NaN,0,systemic lupus erythematosus,premarin and provera,[H][C@@]12CC[C@](OC(C)=O)(C(C)=O)[C@@]1(C)CC[C...,\n Inclusion Criteria:\r\n\r\n ...


In [ ]:
data['criteria'] = data['criteria'].str.strip()

data['criteria'] = data['criteria'].str.replace('\n', '')

In [ ]:
data.head()

,status,why_stop,label,diseases,drugs,smiless,criteria
0,completed,NaN,1,alzheimer disease,galantamine,[H][C@]12C[C@@H](O)C=C[C@]11CCN(C)CC3=C1C(O2)=...,Inclusion Criteria:\r\r - Probable A...
1,completed,NaN,1,alzheimer disease,"donepezil, vitamin e",O=S(=O)(C1=CC=CC=C1)C1=CN=C2C(C=CC=C2N2CCNCC2)...,Inclusion Criteria:\r\r - Memory com...
2,completed,NaN,0,"alzheimer disease, cognition disorders",rivastigmine,CCN(C)C(=O)OC1=CC=CC(=C1)[C@H](C)N(C)C,Inclusion Criteria:\r\r - Are aged 5...
3,completed,NaN,1,depression,imipramine hydrochloride,CN(C)CCCN1C2=CC=CC=C2CCC2=CC=CC=C12,Inclusion Criteria:\r\r - Patient mu...
4,terminated,NaN,0,systemic lupus erythematosus,premarin and provera,[H][C@@]12CC[C@](OC(C)=O)(C(C)=O)[C@@]1(C)CC[C...,Inclusion Criteria:\r\r - Female\r\r...


In [ ]:
data['criteria'] = data['criteria'].str.replace('\r', '')

In [ ]:
data.head()

,status,why_stop,label,diseases,drugs,smiless,criteria
0,completed,NaN,1,alzheimer disease,galantamine,[H][C@]12C[C@@H](O)C=C[C@]11CCN(C)CC3=C1C(O2)=...,Inclusion Criteria: - Probable Alzhe...
1,completed,NaN,1,alzheimer disease,"donepezil, vitamin e",O=S(=O)(C1=CC=CC=C1)C1=CN=C2C(C=CC=C2N2CCNCC2)...,Inclusion Criteria: - Memory complai...
2,completed,NaN,0,"alzheimer disease, cognition disorders",rivastigmine,CCN(C)C(=O)OC1=CC=CC(=C1)[C@H](C)N(C)C,Inclusion Criteria: - Are aged 55-85...
3,completed,NaN,1,depression,imipramine hydrochloride,CN(C)CCCN1C2=CC=CC=C2CCC2=CC=CC=C12,Inclusion Criteria: - Patient must b...
4,terminated,NaN,0,systemic lupus erythematosus,premarin and provera,[H][C@@]12CC[C@](OC(C)=O)(C(C)=O)[C@@]1(C)CC[C...,Inclusion Criteria: - Female ...


In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
data['criteria'][0]

"Inclusion Criteria:          -  Probable Alzheimer's disease          -  Mini-Mental State Examination (MMSE) 10-22 and ADAS greater than or equal to 18          -  Alzheimer's Disease Assessment Scale cognitive portion (ADAS-cog-11) score of at least             18          -  Opportunity for Activities of Daily Living          -  Caregiver          -  Subjects who live with or have regular daily visits from a responsible caregiver             (visit frequency: preferably daily but at least 5 days/week). This includes a friend             or relative or paid personnel. The caregiver should be capable of assisting with the             subject's medication, prepared to attend with the subject for assessments, and willing             to provide information about the subject.        Exclusion Criteria:          -  Conditions that could confound diagnosis          -  Neurodegenerative disorders          -  Acute cerebral trauma          -  Psychiatric disease          -  More than one inf

In [ ]:
data['criteria'] = data['criteria'].apply(lambda x: x.strip() if isinstance(x, str) else x)

In [ ]:
data['criteria']

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [ ]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 4.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
data['why_stop'].value_counts()

why_stop
\n    slow accrual\r\n                                                                                                                                             210
\n    low accrual\r\n                                                                                                                                              150
\n    poor accrual\r\n                                                                                                                                              82
\n    lack of accrual\r\n                                                                                                                                           51
\n    lack of efficacy\r\n                                                                                                                                          40
                                                                                                                                                            

In [ ]:
from transformers import BertTokenizer, BertModel
import torch
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
data.isnull().sum()

status          0
why_stop    13990
label           0
diseases        0
drugs           0
smiless         0
criteria        2
dtype: int64

In [ ]:
data['why_stop'] = data['why_stop'].fillna('no information')

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel

# Example data
texts = data['why_stop'].tolist()

# Define the model and tokenizer
model_name = 'bert-base-uncased'  # Change this to your model name
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(DEVICE)

# Function to process a batch of texts
def process_batch(batch_texts):
    inputs = tokenizer(batch_texts, return_tensors='pt', padding=True, truncation=True).to(DEVICE)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state
    sentence_embeddings = torch.mean(embeddings, dim=1)
    return sentence_embeddings

# Process texts in batches
batch_size = 10  # Adjust the batch size based on your memory constraints
all_embeddings = []

for i in range(0, len(texts), batch_size):
    batch_texts = texts[i:i + batch_size]
    batch_embeddings = process_batch(batch_texts)
    all_embeddings.append(batch_embeddings)

# Concatenate all embeddings
all_embeddings = torch.cat(all_embeddings)

print(all_embeddings.shape)  # Check the shape of the resulting embeddings


torch.Size([17614, 768])


In [ ]:
from sentence_transformers import SentenceTransformer
import pandas as pd
from tqdm import tqdm, trange

# Extract sentences from the 'why_stop' column
sentences = data['criteria'].tolist()

# Load a pretrained Sentence Transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Calculate embeddings by calling model.encode()
embeddings = model.encode(sentences)

# Print the shape of embeddings
print(embeddings.shape)  # This will print the shape of the embeddings array

# Calculate the embedding similarities
similarities = model.similarity(embeddings, embeddings)
print(similarities)

(17614, 384)
tensor([[1.0000, 0.7230, 0.6776,  ..., 0.5735, 0.4637, 0.4156],
        [0.7230, 1.0000, 0.6834,  ..., 0.4470, 0.3556, 0.3678],
        [0.6776, 0.6834, 1.0000,  ..., 0.5252, 0.3604, 0.3897],
        ...,
        [0.5735, 0.4470, 0.5252,  ..., 1.0000, 0.5683, 0.4086],
        [0.4637, 0.3556, 0.3604,  ..., 0.5683, 1.0000, 0.4571],
        [0.4156, 0.3678, 0.3897,  ..., 0.4086, 0.4571, 1.0000]])


In [ ]:
data.head()

status        why_stop  label                                diseases  \
0   completed  no information      1                       alzheimer disease   
1   completed  no information      1                       alzheimer disease   
2   completed  no information      0  alzheimer disease, cognition disorders   
3   completed  no information      1                              depression   
4  terminated  no information      0            systemic lupus erythematosus   

                      drugs  \
0               galantamine   
1      donepezil, vitamin e   
2              rivastigmine   
3  imipramine hydrochloride   
4      premarin and provera   

                                                                                                                  smiless  \
0                                                                 [H][C@]12C[C@@H](O)C=C[C@]11CCN(C)CC3=C1C(O2)=C(OC)C=C3   
1  O=S(=O)(C1=CC=CC=C1)C1=CN=C2C(C=CC=C2N2CCNCC2)=C1, CC(C)CCC[C@@H](C)CCC[C@@H](C)CCC[C@]1(C)CCC2=C(O1)C(C)=C(C)C(O)=C2C   
2                                                                                  CCN(C)C(=O)OC1=CC=CC(=C1)[C@H](C)N(C)C   
3                                                                                     CN(C)CCCN1C2=CC=CC=C2CCC2=CC=CC=C12   
4                           [H][C@@]12CC[C@](OC(C)=O)(C(C)=O)[C@@]1(C)CC[C@@]1([H])[C@@]2([H])C[C@H](C)C2=CC(=O)CC[C@]12C   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [ ]:
print(all_embeddings.shape)  # For the first embedding method
print(embeddings.shape)  # For the second embedding method

torch.Size([17614, 768])
(17614, 384)


In [ ]:
# data.drop(columns=['why_stop', 'criteria'], axis=1, inplace=True)

In [ ]:
data.head()

status        why_stop  label                                diseases  \
0   completed  no information      1                       alzheimer disease   
1   completed  no information      1                       alzheimer disease   
2   completed  no information      0  alzheimer disease, cognition disorders   
3   completed  no information      1                              depression   
4  terminated  no information      0            systemic lupus erythematosus   

                      drugs  \
0               galantamine   
1      donepezil, vitamin e   
2              rivastigmine   
3  imipramine hydrochloride   
4      premarin and provera   

                                                                                                                  smiless  \
0                                                                 [H][C@]12C[C@@H](O)C=C[C@]11CCN(C)CC3=C1C(O2)=C(OC)C=C3   
1  O=S(=O)(C1=CC=CC=C1)C1=CN=C2C(C=CC=C2N2CCNCC2)=C1, CC(C)CCC[C@@H](C)CCC[C@@H](C)CCC[C@]1(C)CCC2=C(O1)C(C)=C(C)C(O)=C2C   
2                                                                                  CCN(C)C(=O)OC1=CC=CC(=C1)[C@H](C)N(C)C   
3                                                                                     CN(C)CCCN1C2=CC=CC=C2CCC2=CC=CC=C12   
4                           [H][C@@]12CC[C@](OC(C)=O)(C(C)=O)[C@@]1(C)CC[C@@]1([H])[C@@]2([H])C[C@H](C)C2=CC(=O)CC[C@]12C   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [ ]:
status_node_values = data['status'].value_counts()

# Print all the column values
print(status_node_values)

status
completed                 12369
terminated                 3548
unknown status              624
active, not recruiting      614
withdrawn                   406
recruiting                   30
suspended                    22
not yet recruiting            1
Name: count, dtype: int64


In [ ]:
drugs = data['drugs'].values
diseases = data['diseases'].values
smiles = data['smiless'].values
labels = data['label'].values
statuses = data['status'].values

In [ ]:
import networkx as nx

G = nx.DiGraph()

for drug,disease,smiles,label,status in zip(drugs, diseases, smiles, labels, statuses):
  for _drug in str(drug).split(','):
    G.add_node(str(_drug), color="red")
  for _disease in str(disease).split(','):
    G.add_node(str(_disease), color="green")
  for _smile in str(smiles).split(','):
    G.add_node(str(_smile), color="blue")
  G.add_node(str(label), color="yellow")
  G.add_edge(str(drug), str(disease), edge_type='heals')
  G.add_edge(str(smiles), str(drug), edge_type='builds')
  G.add_edge(str(disease), str(label), edge_type='has')

  status_node = f"{status}{drug}_{disease}"
  G.add_node(status_node, node_type='status')
  G.add_edge(drug, status_node, edge_type='has_status')
  G.add_edge(status_node, disease, edge_type='status_of')

In [ ]:
!pip install pyvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 60.0 MB/s eta 0:00:00


In [ ]:
from pyvis.network import Network
net = Network(notebook=True, cdn_resources="in_line", directed=True)
net.from_nx(G)
net.show("example.html")

example.html


In [ ]:
import random

# Determine the number of nodes and edges to include in the subgraph (adjust as needed)
subset_size = int(len(G) * 0.1)  # 10% of the original graph

# Get a random subset of nodes
subset_nodes = random.sample(G.nodes(), subset_size)

# Create a subgraph from the original graph based on the subset of nodes
subgraph = G.subgraph(subset_nodes).copy()

# Visualize the subgraph using pyvis
from pyvis.network import Network

net = Network(notebook=True, directed=True, cdn_resources='in_line')
net.from_nx(subgraph)
net.show("example_subgraph.html")

<ipython-input-40-9604080bdffc>:7: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  subset_nodes = random.sample(G.nodes(), subset_size)


example_subgraph.html
